# Task 1
## 2

After you have found the best-performing hyperparameter sets, take the 3 best onesand train new models on the CIFAR-10 dataset to see whether your performance gains translate to a different dataset.

In [1]:
# Importing libraries
import tensorflow as tf
from tensorflow import keras
import sklearn
from sklearn.datasets import load_sample_image
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Lets first import the data

In [4]:
cifar10 = keras.datasets.cifar10

In [5]:
# Creating the train and test sets
(X_train_full, y_train_full), (X_test, y_test) = cifar10.load_data()

print(X_train_full.shape)

(50000, 32, 32, 3)


When we compare this to the fashion_MNIST dataset we can see the following:
- the dimensions of each image is bigger
- it has 3 channels instead of one

In [6]:
batch_size, height, width, channels = X_train_full.shape
input_shape = (height, width, channels)

# Normalising the pixel values to the range [0, 1]
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0

# Creating a validation set of 10%
validation_size = int(0.1 * len(X_train_full))

X_valid, X_train = X_train_full[:validation_size], X_train_full[validation_size:]
y_valid, y_train = y_train_full[:validation_size], y_train_full[validation_size:]

In [7]:
# For more efficient code we"ll change the data types to float32
X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')
X_test = X_test.astype('float32')

We'll first try to run the final MLP model on this dataset. For the fashion_mnist dataset this model had an accuracy of 88.2% and a loss of 33.2% on the test set.

In [8]:
model_mlp = keras.models.Sequential()
model_mlp.add(keras.layers.Flatten(input_shape = input_shape)) # adjusted input shape

model_mlp.add(keras.layers.Dense(300))
model_mlp.add(keras.layers.LeakyReLU(alpha = 0.1))
model_mlp.add(keras.layers.Dropout(0.5))

model_mlp.add(keras.layers.Dense(200))
model_mlp.add(keras.layers.LeakyReLU(alpha = 0.1))
model_mlp.add(keras.layers.Dropout(0.5))

model_mlp.add(keras.layers.Dense(100))
model_mlp.add(keras.layers.LeakyReLU(alpha = 0.1))
model_mlp.add(keras.layers.Dropout(0.5))

model_mlp.add(keras.layers.Dense(10, activation="softmax"))


learning_rate = 0.01
model_mlp.compile(loss = "sparse_categorical_crossentropy", optimizer=keras.optimizers.SGD(learning_rate = learning_rate), metrics=["accuracy"])

early_stopping = keras.callbacks.EarlyStopping(patience= 7, restore_best_weights= True) 
history = model_mlp.fit(X_train, y_train, epochs=70, # try 70
                    validation_data=(X_valid, y_valid),
                    callbacks = [early_stopping]) 

# final score
score_all = model_mlp.evaluate(X_test, y_test, verbose=0)
best_epoch = np.argmin(history.history['val_loss']) + 1
print('epochs used:', best_epoch)
print('Test loss:', score_all[0])
print('Test accuracy:', score_all[1])

C:\Users\hsuik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\hsuik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/70
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.1445 - loss: 2.2809 - val_accuracy: 0.2512 - val_loss: 2.0319
Epoch 2/70
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.2225 - loss: 2.0813 - val_accuracy: 0.3316 - val_loss: 1.9067
Epoch 3/70
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.2627 - loss: 2.0054 - val_accuracy: 0.3436 - val_loss: 1.8800
Epoch 4/70
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.2814 - loss: 1.9548 - val_accuracy: 0.3372 - val_loss: 1.8452
Epoch 5/70
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.2990 - loss: 1.9237 - val_accuracy: 0.3702 - val_loss: 1.8013
Epoch 6/70
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.3140 - loss: 1.8935 - val_accuracy: 0.3668 - val_loss: 1.7764
Epoch 7/70
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.3205 - loss: 1.8641 - val_accuracy: 0.3834 - val_loss: 1.7509
Epoch 8/70
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.3361 - loss: 1.8433 - 

The accuracy seems to still be increasing so lets add 30 more epochs

In [11]:
history = model_mlp.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks = [early_stopping]) 

# final score
score_all = model_mlp.evaluate(X_test, y_test, verbose=0)
best_epoch = np.argmin(history.history['val_loss']) + 1
print('epochs used:', best_epoch)
print('Test loss:', score_all[0])
print('Test accuracy:', score_all[1])

Epoch 1/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.4718 - loss: 1.4832 - val_accuracy: 0.5026 - val_loss: 1.4069
Epoch 2/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.4739 - loss: 1.4633 - val_accuracy: 0.5022 - val_loss: 1.3944
Epoch 3/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.4715 - loss: 1.4771 - val_accuracy: 0.4898 - val_loss: 1.4215
Epoch 4/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.4740 - loss: 1.4711 - val_accuracy: 0.5118 - val_loss: 1.3905
Epoch 5/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.4751 - loss: 1.4671 - val_accuracy: 0.4890 - val_loss: 1.4259
Epoch 6/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.4728 - loss: 1.4646 - val_accuracy: 0.4860 - val_loss: 1.4090
Epoch 7/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.4780 - loss: 1.4655 - val_accuracy: 0.4954 - val_loss: 1.3941
Epoch 8/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.4804 - loss: 1.4549 - 

This is quite a bad score. The model's performance does not seem to translate to the CIFAR10 dataset. This could be because of the fact that this CIFAR10 dataset is a lot more complex than the fashion_MNIST dataset. The MLP model therefore most likely is not complex enough due to insufficient layers. Also the model was made with data with only 1 channel in mind, the model now flattens CIFAR10's imagages. This results in a loss of information on the arrangement of the pixels. This makes it very hard for the model to recognize patterns.

Now let's also try the best CNN model. 

In [9]:
model_cnn1 = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation="relu", padding="same",
                        input_shape = input_shape), # No stride because images are not very large
    keras.layers.MaxPooling2D(2), # divides each spatial dimension by 2
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"), # double filters
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2), # divide dimensions by 2
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"), # again double filters
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2), 
    # fully connected network:
    keras.layers.Flatten(), # needs to be 1D for dense network
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5), # droupout of 50% to prevent overfitting
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax") # output layer
])

model_cnn1.compile(loss = "sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

model_cnn1.fit(X_train, y_train, batch_size=64, epochs=12, validation_data = (X_valid, y_valid))

C:\Users\hsuik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 172s 242ms/step - accuracy: 0.1219 - loss: 2.2886 - val_accuracy: 0.1248 - val_loss: 2.2764
Epoch 2/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 192s 273ms/step - accuracy: 0.1937 - loss: 2.1543 - val_accuracy: 0.1326 - val_loss: 2.8423
Epoch 3/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 211s 300ms/step - accuracy: 0.2402 - loss: 2.0566 - val_accuracy: 0.3304 - val_loss: 1.8589
Epoch 4/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 220s 312ms/step - accuracy: 0.2807 - loss: 1.9451 - val_accuracy: 0.3632 - val_loss: 1.7788
Epoch 5/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 209s 296ms/step - accuracy: 0.3186 - loss: 1.8541 - val_accuracy: 0.1938 - val_loss: 2.3564
Epoch 6/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 219s 310ms/step - accuracy: 0.3397 - loss: 1.7846 - val_accuracy: 0.4320 - val_loss: 1.6229
Epoch 7/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 213s 303ms/step - accuracy: 0.3732 - loss: 1.7135 - val_accuracy: 0.4532 - val_loss: 1.5524
Epoch 8/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 226s 321ms/step - accuracy: 0.3972 -

lets do 12 more

In [12]:
model_cnn1.fit(X_train, y_train, batch_size=64, epochs=12, validation_data = (X_valid, y_valid))

Epoch 1/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 148s 210ms/step - accuracy: 0.4765 - loss: 1.4557 - val_accuracy: 0.4732 - val_loss: 1.4899
Epoch 2/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 182s 259ms/step - accuracy: 0.4919 - loss: 1.4189 - val_accuracy: 0.3920 - val_loss: 1.6465
Epoch 3/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 211s 299ms/step - accuracy: 0.5130 - loss: 1.3672 - val_accuracy: 0.5084 - val_loss: 1.4694
Epoch 4/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 206s 292ms/step - accuracy: 0.5222 - loss: 1.3439 - val_accuracy: 0.5220 - val_loss: 1.3723
Epoch 5/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 227s 323ms/step - accuracy: 0.5372 - loss: 1.3018 - val_accuracy: 0.4480 - val_loss: 1.5795
Epoch 6/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 235s 333ms/step - accuracy: 0.5499 - loss: 1.2811 - val_accuracy: 0.5848 - val_loss: 1.1720
Epoch 7/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 215s 305ms/step - accuracy: 0.5676 - loss: 1.2214 - val_accuracy: 0.5034 - val_loss: 1.4022
Epoch 8/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 210s 298ms/step - accuracy: 0.5852 -

In [13]:
score = model_cnn1.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.064365267753601
Test accuracy: 0.6254000067710876


This is a big improvement compared to the MLP model. However, compared to the models performance on the fashion_mnist dataset it scores bad....

Lastly, let's try the second best CNN model. This was the model with a decreased value for the second dropout layer.

In [15]:
model_cnn2 = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation="relu", padding="same",
                        input_shape = input_shape), # No stride because images are not very large
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"), 
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2), 
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"), 
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2), 
    # fully connected network:
    keras.layers.Flatten(), 
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5), 
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.25), # decreased dropout
    keras.layers.Dense(10, activation="softmax") # output layer
])

model_cnn2.compile(loss="sparse_categorical_crossentropy", 
                optimizer="sgd", 
                metrics=['accuracy'])

model_cnn2.fit(X_train, y_train, batch_size=64, epochs=12, validation_data=(X_valid, y_valid)) # only 3 to see if it performs similar in the beginning to previous models

Epoch 1/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 151s 214ms/step - accuracy: 0.1266 - loss: 2.2914 - val_accuracy: 0.2404 - val_loss: 2.1079
Epoch 2/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 191s 272ms/step - accuracy: 0.2155 - loss: 2.1210 - val_accuracy: 0.1488 - val_loss: 2.5503
Epoch 3/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 210s 298ms/step - accuracy: 0.2861 - loss: 1.9624 - val_accuracy: 0.3108 - val_loss: 1.8524
Epoch 4/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 167s 237ms/step - accuracy: 0.3428 - loss: 1.8122 - val_accuracy: 0.4128 - val_loss: 1.6062
Epoch 5/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 172s 245ms/step - accuracy: 0.3763 - loss: 1.7151 - val_accuracy: 0.4048 - val_loss: 1.6291
Epoch 6/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 1486s 2s/step - accuracy: 0.4081 - loss: 1.6319 - val_accuracy: 0.3552 - val_loss: 1.7114
Epoch 7/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 184s 261ms/step - accuracy: 0.4266 - loss: 1.5799 - val_accuracy: 0.4510 - val_loss: 1.5009
Epoch 8/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 163s 232ms/step - accuracy: 0.4466 - l

Right now it seems to do better than the previous models at epoch 12. Let's do 12 more.

In [17]:
model_cnn2.fit(X_train, y_train, batch_size=64, epochs=12, validation_data=(X_valid, y_valid)) 

Epoch 1/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 146s 207ms/step - accuracy: 0.5345 - loss: 1.3017 - val_accuracy: 0.5524 - val_loss: 1.2276
Epoch 2/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 180s 255ms/step - accuracy: 0.5504 - loss: 1.2665 - val_accuracy: 0.5832 - val_loss: 1.1492
Epoch 3/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 152s 216ms/step - accuracy: 0.5690 - loss: 1.2187 - val_accuracy: 0.5142 - val_loss: 1.3589
Epoch 4/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 179s 254ms/step - accuracy: 0.5780 - loss: 1.1887 - val_accuracy: 0.6006 - val_loss: 1.1390
Epoch 5/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 156s 222ms/step - accuracy: 0.5944 - loss: 1.1522 - val_accuracy: 0.6016 - val_loss: 1.1433
Epoch 6/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 179s 254ms/step - accuracy: 0.6099 - loss: 1.1139 - val_accuracy: 0.5688 - val_loss: 1.2289
Epoch 7/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 157s 223ms/step - accuracy: 0.6311 - loss: 1.0567 - val_accuracy: 0.5576 - val_loss: 1.3043
Epoch 8/12
704/704 ━━━━━━━━━━━━━━━━━━━━ 178s 253ms/step - accuracy: 0.6374 -

In [18]:
score = model_cnn2.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.9939793944358826
Test accuracy: 0.6536999940872192


Compared to the other CNN model after 24 epochs this model scored a lot better, especially looking at the loss. Both models still seem to be improving with more epochs. Overall the CNN models seemed to perfrom better than the MLP model. This could be due to the fact that ...